In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import math

In [64]:
metrics_to_analyze = ["frame_count", "ear", "mar", "yaw", "pitch", "pitch1", "pitch2", "nose_tip_y"]
df = pd.read_csv("NTHUDDD_dataset2/train/012_noglasses_slowBlinkWithNodding.csv")[metrics_to_analyze]
df_nonsleepy = pd.read_csv("NTHUDDD_dataset2/train/032_noglasses_sleepy.csv")[metrics_to_analyze]

In [65]:
pitch_values = df["pitch1"].values
thresholds_dinamicos = []
upper_thresholds_dinamicos = []
lower_thresholds_dinamicos = []
pitch_suavizados = []

num_past_frames = 300
gray_zone_perc = 0.02
th_perc = 0.92
th = 70
frames_suavizado_pitch = 30

last_pos_pitch_values = []
last_pitch_values = []
previous_headnod_state = None

headnod_set = []
headnod_intervals = []

for ind, pitch in enumerate(pitch_values):
    if math.isnan(pitch):
        thresholds_dinamicos.append(thresholds_dinamicos[-1])
        lower_thresholds_dinamicos.append(lower_thresholds_dinamicos[-1])
        upper_thresholds_dinamicos.append(upper_thresholds_dinamicos[-1])
        pitch_suavizados.append(0)
        continue
    
    gray_zone = 0
    if last_pos_pitch_values != [] and num_past_frames > 0:
        media_pitch = sum(last_pos_pitch_values)/len(last_pos_pitch_values)
        th = th_perc * media_pitch
        gray_zone = gray_zone_perc * media_pitch

    mean_pitch = pitch
    if last_pitch_values != [] and frames_suavizado_pitch > 0:
        pitch_last = min(frames_suavizado_pitch, len(last_pitch_values))
        pitch_values_to_analyze = last_pitch_values[-pitch_last:]
        mean_pitch = (sum(pitch_values_to_analyze) + pitch) / (pitch_last + 1)

    upper_threshold = th + gray_zone
    lower_threshold = th - gray_zone
    if mean_pitch < lower_threshold:
        current_headnod_state = True
    elif mean_pitch > upper_threshold:
        current_headnod_state = False
    else:
        current_headnod_state = mean_pitch <= th
        if previous_headnod_state is not None:
            current_headnod_state = previous_headnod_state

    current_headnod_state = current_headnod_state and mean_pitch < 80
        
    if current_headnod_state:
        headnod_set.append(ind)
    
    if not current_headnod_state and previous_headnod_state:
        headnod_intervals.append(headnod_set)
        headnod_set = []
    previous_headnod_state = current_headnod_state

    if not current_headnod_state and num_past_frames > 0:
        if len(last_pos_pitch_values) >= num_past_frames:
            last_pos_pitch_values.pop(0)
        last_pos_pitch_values.append(mean_pitch)
    
    if frames_suavizado_pitch > 0:
        if len(last_pitch_values) >= frames_suavizado_pitch:
            last_pitch_values.pop(0)
        last_pitch_values.append(pitch)
    
    thresholds_dinamicos.append(th)
    upper_thresholds_dinamicos.append(upper_threshold)
    lower_thresholds_dinamicos.append(lower_threshold)
    pitch_suavizados.append(mean_pitch)
# print(thresholds_dinamicos)
print(headnod_intervals)
print(f"NUM_HEADNODS: {len(headnod_intervals)}")

[[324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394], [708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772], [1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 11

In [66]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["frame_count"], y=df["pitch1"], name="True Pitch Values"))
fig.add_trace(go.Scatter(x=df["frame_count"], y=thresholds_dinamicos, name="TD"))
fig.add_trace(go.Scatter(x=df["frame_count"], y=upper_thresholds_dinamicos, name="UTD"))
fig.add_trace(go.Scatter(x=df["frame_count"], y=lower_thresholds_dinamicos, name="LTD"))
fig.add_trace(go.Scatter(x=df["frame_count"], y=pitch_suavizados, name="SUAV"))

In [6]:
var = "pitch1"
mean_ear = df["ear"].mean()
fig = px.line(df, x="frame_count", y=var, title=f"Evolucion del {var} durante el video")
fig.update_layout(shapes=[
dict(
  type= 'line',
  yref= 'y', y0=70, y1=70,   # adding a horizontal line at Y = 1
  xref= 'paper', x0=0, x1=1
     ) 
])
fig.show()
mean_ear_nonsleepy = df_nonsleepy["ear"].mean()
fig = px.line(df_nonsleepy, x="frame_count", y=var, title=f"Evolucion del {var} durante el video")
fig.update_layout(shapes=[
dict(
  type= 'line',
  yref= 'y', y0=mean_ear_nonsleepy, y1=mean_ear_nonsleepy,   # adding a horizontal line at Y = 1
  xref= 'paper', x0=0, x1=1
     ) 
])
fig.show()